In [1]:
#Load modules
import zarr
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import yaml
from pathlib import Path
import allel

from dask.distributed import Client
import dask
dask.config.set(**{'array.slicing.split_large_chunks': False}) # Silence large chunk warnings
import dask.array as da
from dask import delayed, compute
from dask_gateway import Gateway
import functools
import numcodecs
from fsspec.implementations.zip import ZipFileSystem
from collections.abc import Mapping
import gcsfs
import numba
import psutil
from humanize import naturalsize

import pickle
import platform

import traceback
import logging

from pyprojroot import here
from bokeh.plotting import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator

### Connect to gcs

In [ ]:
gcs = gcsfs.GCSFileSystem()

In [ ]:
gcs.ls('vo_afun_release_master_us_central1')[:3]

### Set up data access

In [4]:
production_root = Path('vo_afun_release_master_us_central1')
vo_afun_staging = Path(production_root, 'v1.0')
sampleset_staging_dir = Path(vo_afun_staging, 'snp_genotypes', 'all')

#Decision tree or static filters
genomic_positions_site_filter_dt_data_cloud_zarr_dir = 'vo_afun_release_master_us_central1/v1.0/site_filters/dt_20200416/funestus'
genomic_positions_site_filter_sc_data_cloud_zarr_dir = 'vo_afun_release_master_us_central1/v1.0/site_filters/sc_20220908/funestus'

repo_clone_path = here()
release_config_path = repo_clone_path / 'tracking' / 'release' / 'v1.0' / 'config.yml'

with open(release_config_path) as fh:
    config = yaml.load(fh, Loader=yaml.BaseLoader)
    
samplesets = config["sample_sets"]

In [5]:
meta = pd.read_csv("../../metadata/supp1_tab2.csv")
meta.columns

Index(['VBS_sample_id', 'contributor', 'country', 'location',
       'collection_year', 'collection_month', 'latitude', 'longitude',
       'date_of_collection', 'collection_context', 'collection_method',
       'collection_time', 'collector_sample_id', 'sample_type',
       'specimen_sex_partner', 'specimen_storage_partner', 'Lifestage',
       'dna_extraction_method_partner', 'sample_type_received',
       'body_part_lab', 'dna_extraction_method_lab', 'total_dna_lab',
       'species_pcr', 'sequencing_pipeline', 'mean_cov', 'median_cov',
       'modal_cov', 'mean_cov_2RL', 'median_cov_2RL', 'mode_cov_2RL',
       'mean_cov_3RL', 'median_cov_3RL', 'mode_cov_3RL', 'mean_cov_X',
       'median_cov_X', 'mode_cov_X', 'frac_gen_cov', 'divergence', 'pc_contam',
       'LLR', 'sex_call', 'PCA_QC_outlier', 'FILTER_frac_genome_cov',
       'FILTER_median_cov', 'FILTER_contamination', 'FILTER_nosexcall',
       'FILTER_divergence', 'FILTER_pca', 'sample_set', 'adm_level',
       'geographic_coh

### Functions

In [6]:
# load a single array from field/chrom/sampleset
# internal path for calldata is chrom/calldata/field
# sampleset_calldata = sampleset_staging_dir / sset
# sampleset is needed to load species spec.
def load_single_field(zarr_path, internal_path):
      
    inz = zarr.group(is_gcloud(zarr_path), overwrite=False)
    
    oo = da.from_zarr(inz[internal_path])  
    
    if oo.ndim == 1:
        oo = oo.reshape((1, -1))
           
    return oo

In [7]:
## General function to concatenate data.
## Selected chunk size may be more appropriate for some than others.
def concatenate_along_axis(base_dir, internal_path, req_samplesets):
    
    # work out shape
    data = [load_single_field(base_dir / ss, internal_path) for ss in req_samplesets]
    
    return da.concatenate(data, axis=1)

In [8]:
def is_gcloud(path):
    
    try: 
        return gcs.get_mapper(path.as_posix())
    except NameError as e:
        return path.as_posix()

In [9]:
def load_filter(chrom, filter_dir = genomic_positions_site_filter_sc_data_cloud_zarr_dir):
    gcsmap = gcs.get_mapper(filter_dir)
    genomic_positions_site_filter_data = zarr.Group(gcsmap, read_only=True)
    filter_pass = da.from_zarr(
            genomic_positions_site_filter_data[chrom]['variants/filter_pass'])
    return filter_pass

In [10]:
def load_position(chrom):
    store = gcs.get_mapper(
        f'gs://vo_afun_release_master_us_central1/v1.0/snp_genotypes/all/sites')
    root = zarr.open(store, mode='r')
    pos = root[chrom]['variants/POS'][:]
    return pos

In [11]:
def read_in_genotypes_positions(chrom, samples_idx, samplesets, posl, posu, \
                                filter_dir = genomic_positions_site_filter_sc_data_cloud_zarr_dir):

    # load the genotypes and positions
    gt_d = concatenate_along_axis(sampleset_staging_dir, f"{chrom}/calldata/GT", samplesets)
    gt = allel.GenotypeDaskArray(gt_d)
    pos = load_position(chrom)
    
    if posu==-1:
        posu = pos.max()+1
        
    if posl==-1:
        posl = pos.min()
    
    #load the filter
    loc_filt = load_filter(chrom, filter_dir)
    
    #filter by positions
    pos_filt = (pos>=posl) & (pos<posu)
    
    #apply the filter to positions and genotypes
    gt = gt.compress((loc_filt) & (pos_filt), axis=0)
    pos = pos[(loc_filt) & (pos_filt)]
    
    #subset to desired samples 
    gt1 = da.take(gt, samples_idx, axis=1)
    
    
    return gt1, pos

In [30]:
def split_to_windows(values, window_size):
    n_windows = len(values)//window_size
    vals = np.zeros(n_windows)
    for w in np.arange(n_windows):
        vals[w] = np.nanmean(values[w*window_size:(w+1)*window_size])
    return vals

In [32]:
def compute_wattersons_theta(chrom, samples_idx, posl, posu, missing_frac, window_size,\
                     samplesets):
    
    #read in genotypes
    gt, pos = read_in_genotypes_positions(chrom, samples_idx, samplesets, posl=posl, posu=posu)
    
    #count alleles 
    ac = gt.count_alleles(max_allele=3)
    
    #filters for missingness, biallelism and maf
    missing_filter = (ac.sum(axis=1) >= (1-missing_frac)*2*len(samples_idx))
    
    #get filtered allele counts
    ac_f = ac.compress(missing_filter, axis=0)
    pos_f = pos[missing_filter]
    
    #find segregating sites
    s = ac_f.is_segregating()
    
    #get number of segregating sites in non-overlapping windows
    S_windowed = split_to_windows(s.compute(), window_size)
    pos_windowed = split_to_windows(pos_f, window_size)
    
    #get starting position new reading frame
    end_pos = pos_f[len(S_windowed)*window_size]

    return S_windowed, pos_windowed, end_pos

In [14]:
def loop_through_reading_frames(chrom, samples_idx, posmax, posmin, samplesets, \
                                reading_frame_size, missing_frac, window_size):
    if posmax == -1:
        posmax = load_position(chrom).max()
    
    S_win, pos_win, rf_start = compute_wattersons_theta(chrom, samples_idx, posmin, min(posmin+reading_frame_size, posmax), \
                                        missing_frac=missing_frac, samplesets=samplesets, window_size=window_size)
    rf_end = rf_start + 1
    while rf_end < posmax:
        rf_end = min(rf_start + reading_frame_size, posmax)
        S_win_rf, pos_win_rf, rf_start = compute_wattersons_theta(chrom, samples_idx, rf_start, rf_end, \
                                    missing_frac=missing_frac, samplesets=samplesets, window_size=window_size)
        S_win = np.concatenate((S_win, S_win_rf))
        pos_win = np.concatenate((pos_win, pos_win_rf))
        
    return S_win, pos_win
        
        

In [15]:
def compute_watterson_all_cohorts(chrom, ordered_cohorts, meta, 
                            posmin=-1, posmax=-1, samplesets=samplesets, 
                          reading_frame_size=10_000_000, missing_frac=0.05, 
                                 window_size=100_000):
    
    for cohort in ordered_cohorts:
        samples_idx = meta.loc[(meta['geographic_cohort_code']==cohort) & (meta.subset_3=='Y')].index
        S_win, pos_win = loop_through_reading_frames(chrom, \
                                    samples_idx, posmin, posmax, samplesets,\
                                    reading_frame_size, missing_frac, window_size=window_size)
        np.save(f'thetas/{cohort}_{chrom}_theta.npy', S_win)
        np.save(f'thetas/{cohort}_{chrom}_pos.npy', pos_win)
        print(f"On {chrom} cohort {cohort} done")
        
    

In [16]:
#put meta in order as dask is stored
sample_order = concatenate_along_axis(sampleset_staging_dir, "samples", samplesets).compute()
sample_order = (sample_order[0]).astype(str)
meta.set_index('VBS_sample_id', inplace=True)
meta = meta.loc[sample_order]
meta.reset_index(inplace=True)
meta.head()

,VBS_sample_id,contributor,country,location,collection_year,collection_month,latitude,longitude,date_of_collection,collection_context,...,mismatches,error_rate,average_quality,insert_size_average,insert_size_standard_deviation,inward_oriented_pairs,outward_oriented_pairs,pairs_with_other_orientation,pairs_on_different_chromosomes,ena_run_accessions
0,VBS24195,Samuel Dadzie,Ghana,Dimabi,2017,8,9.42011,-1.082560,13/8/2017,Indoor,...,347337836,0.0253,38.0,449.2,980.3,40744336,3027720,261505,2387067,"ERR3669145,ERR3669181,ERR3669217"
1,VBS24196,Samuel Dadzie,Ghana,Gbullung,2017,7,9.48792,-1.009220,16/7/2017,Outdoor,...,310727330,0.0267,37.0,543.9,1093.7,35313760,1821441,274916,2514096,"ERR3669218,ERR3682461,ERR3682496"
2,VBS24197,Samuel Dadzie,Ghana,Dimabi,2017,7,9.42011,-1.082560,16/7/2017,Outdoor,...,285245941,0.0291,37.4,589.7,1147.3,31196598,783227,325353,1247119,"ERR3669219,ERR3682462,ERR3682497"
3,VBS24198,Samuel Dadzie,Ghana,Dimabi,2017,8,9.42011,-1.082560,13/8/2017,Indoor,...,249061148,0.0280,37.4,500.9,1158.2,26472975,2023471,227133,1738727,"ERR3669220,ERR3682463,ERR3682498"
4,VBS24199,Samuel Dadzie,Ghana,Gupanarigu,2017,8,9.49672,-0.952220,13/8/2017,Indoor,...,361355171,0.0262,37.4,462.2,1022.8,41241829,2997300,279985,2520261,"ERR3669221,ERR3682464,ERR3682499"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,VBS24534,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.85137,40.594441,20/8/2015,Unknown,...,313096144,0.0256,38.3,544.9,1088.8,38769216,1007181,350264,1446714,"ERR3669140,ERR3669176,ERR3669212"
652,VBS24535,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.85137,40.594441,20/8/2015,Unknown,...,324338224,0.0237,38.0,532.7,986.2,43365369,1029280,322255,1725204,"ERR3669141,ERR3669177,ERR3669213"
653,VBS24536,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.85137,40.594441,20/8/2015,Unknown,...,308174540,0.0257,37.9,586.7,1068.1,38032378,1069278,329066,1520472,"ERR3669142,ERR3669178,ERR3669214"
654,VBS24537,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.85137,40.594441,20/8/2015,Unknown,...,323179760,0.0254,37.9,589.5,1076.5,40549553,999904,344766,1591309,"ERR3669143,ERR3669179,ERR3669215"


### Set up dask cluster

In [17]:
gateway = Gateway()
gateway.list_clusters()

[]

In [18]:
#gateway = Gateway()
conda_prefix = os.environ["CONDA_PREFIX"]
current_environment = 'global/'+conda_prefix.split('/')[5]
cluster = gateway.new_cluster(
    profile='standard', 
    conda_environment = current_environment,
)
cluster

In [19]:
client=cluster.get_client()

In [20]:
cluster.scale(20)

In [21]:
cohorts_ordered = ['GH-N', 'BJ', 'GH-A', 'NG', 'CM', 'CD-H', 'UG', 'KE-N',
              'GA', 'CD-K', 'MZ-C', 'ZM', 'MZ-M']

In [ ]:
compute_watterson_all_cohorts(chrom='2RL', ordered_cohorts=cohorts_ordered, 
                        meta=meta)

/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On 2RL cohort GH-N done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On 2RL cohort BJ done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

...

/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On 2RL cohort MZ-C done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On 2RL cohort ZM done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 98.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

In [ ]:
compute_watterson_all_cohorts(chrom='3RL', ordered_cohorts=cohorts_ordered, 
                        meta=meta)

/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 32.22 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On 3RL cohort GH-N done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On 3RL cohort BJ done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

...

/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On 3RL cohort MZ-C done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On 3RL cohort ZM done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 80.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

In [35]:
compute_watterson_all_cohorts(chrom='X', ordered_cohorts=cohorts_ordered, 
                        meta=meta)

/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 60.92 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 21.30 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On X cohort GH-N done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 21.30 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 21.30 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On X cohort BJ done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 21.30 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 21.30 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

...

On X cohort MZ-C done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 21.30 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 21.30 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On X cohort ZM done


/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 21.30 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/global/cd82d5398c0ffba89cbbb97f8c070d3e0bf1dbf28105a9a5d51458fa46f436d2-20250120-115320-589802-82-mgenv-7.0.0/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 21.30 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for mor

On X cohort MZ-M done


In [36]:
cluster.shutdown()

In [17]:
for report in gateway.list_clusters():
    gateway.connect(report.name).shutdown()